## Pandas ++
[referencia rapida](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_r.html#quick-reference)

In [ ]:
# vamos pensar em manipulação de dados fazendo um paralelo com o dplyr.
# dplyr: 6 verbos principais
# select()    # seleciona colunas do data.frame
# arrange()   # reordena as linhas do data.frame
# filter()    # filtra linhas do data.frame
# mutate()    # cria novas colunas no data.frame (ou atualiza as colunas existentes)
# summarise() + group_by() # sumariza o data.frame
# left_join   # junta dois data.frames

### Exercícios

Os exercícios abaixo consideram a base do IMDB.

In [2]:
import pandas as pd

In [ ]:
# Objetivo: descobrir qual o filme mais caro, 
# e com melhor nota dos anos 2000

In [3]:
dados = pd.read_csv("/workspaces/main-python-r/dados/imdb.csv")

dados.head(1)

,id_filme,titulo,ano,data_lancamento,generos,duracao,pais,idioma,orcamento,receita,receita_eua,nota_imdb,num_avaliacoes,direcao,roteiro,producao,elenco,descricao,num_criticas_publico,num_criticas_critica
0,tt0023352,Prestige,1931.0,1932-01-22,"Adventure, Drama",71,USA,English,NaN,NaN,NaN,5.7,240,Tay Garnett,"Harry Hervey, Tay Garnett",RKO Pathé Pictures,"Ann Harding, Adolphe Menjou, Melvyn Douglas, I...",A woman travels to a French penal colony in In...,12.0,2.0


In [14]:
dados_orcamento = dados.sort_values(["orcamento"], ascending=False).filter(['titulo', 'orcamento', 'data_lancamento'])

In [9]:
dados_anos_2000 = dados.query("ano >= 2000 & ano < 2010")

In [ ]:
# Objetivo: pegar todos os filmes que sejam do genero "Comedy"

In [ ]:
dados[dados['generos'].str.contains("comedy", case = False)]

### Mutate

In [3]:
import pandas as pd

imdb = pd.read_csv("../dados/imdb.csv")
imdb_simples = imdb.filter(["titulo", "ano"])

In [8]:
imdb_simples.assign(
    coluna_nova = imdb_simples.filter(['ano']) < 1945,
    # não é muito recomendado fazer isso ^
    coluna_nova2 = imdb_simples.ano < 1945,
    # esse é o jeito elgal de criar essa coluna ^
    coluna_numero = imdb_simples.ano-1900
)

,titulo,ano,coluna_nova,coluna_nova2,coluna_numero
0,Prestige,1931.0,True,True,31.0
1,Nob Hill,1945.0,False,False,45.0
2,The Shade,1999.0,False,False,99.0
3,Viewer Discretion Advised,1998.0,False,False,98.0
4,Broadcast News,1987.0,False,False,87.0
...,...,...,...,...,...
28485,Jumanji: The Next Level,2019.0,False,False,119.0
28486,Monster on the Campus,1958.0,False,False,58.0
28487,The Ward,2010.0,False,False,110.0
28488,A Game of Death,1945.0,False,False,45.0


In [9]:
(imdb.assign(
    lucro = lambda x: x.receita-x.orcamento,
    # essa linha é igual a trocar o x. por imdb. e tirar o "lambda x"
    # x. vira imdb., então:  x.receita-x.orcamento vira imdb.receita-imdb.orcamento
    categoria_lucrou = lambda x: x.lucro > 0,
    duracao = imdb.duracao/60
).
 filter(["titulo","ano", "lucro", "duracao", "categoria_lucrou"]))

,titulo,ano,lucro,duracao,categoria_lucrou
0,Prestige,1931.0,NaN,1.183333,False
1,Nob Hill,1945.0,NaN,1.583333,False
2,The Shade,1999.0,NaN,1.383333,False
3,Viewer Discretion Advised,1998.0,NaN,1.750000,False
4,Broadcast News,1987.0,47331309.0,2.216667,True
...,...,...,...,...,...
28485,Jumanji: The Next Level,2019.0,671575993.0,2.050000,True
28486,Monster on the Campus,1958.0,NaN,1.283333,False
28487,The Ward,2010.0,-4656180.0,1.483333,False
28488,A Game of Death,1945.0,NaN,1.200000,False


In [10]:
imdb2 = imdb
imdb2['lucrou'] = imdb2['receita'] > imdb2['orcamento']
imdb2.filter(["lucrou"])

,lucrou
0,False
1,False
2,False
3,False
4,True
...,...
28485,True
28486,False
28487,False
28488,False


In [12]:
imdb = imdb.drop(["lucrou"], axis = "columns")

In [14]:
imdb2 = imdb.copy()
imdb2['lucrou'] = imdb2['receita'] > imdb2['orcamento']
imdb2.filter(["lucrou"])

,lucrou
0,False
1,False
2,False
3,False
4,True
...,...
28485,True
28486,False
28487,False
28488,False


In [38]:
# case when
import numpy as np
lucrou_df = imdb.assign(
    lucro = lambda x: x["receita"] - x['orcamento'],
    lucrou_bastante = lambda x: np.select(
        [x.lucro >  1000000, x.lucro > 0, x.lucro.isnull()], 
        ["Mais de milhão", "Pouco", "Sem info"], 
        "Não lucrou"
    )
).filter(["lucro", "lucrou_bastante"])

In [40]:
lucrou_df.lucrou_bastante.value_counts()

Sem info          23758
Mais de milhão     2403
Não lucrou         2148
Pouco               181
Name: lucrou_bastante, dtype: int64

### Exercícios

In [ ]:
# Obter um dataframe com os filmes que deram prejuizo.

In [16]:
imdb = pd.read_csv("/workspaces/main-python-r/dados/imdb.csv")

In [60]:
imdb_prejuizo = imdb.assign(
    lucro = lambda x: x.receita - x.orcamento
).query("lucro < 0")
imdb_prejuizo.filter(["titulo", "lucro"])

,titulo,lucro
26,The Blob,-10752057.0
44,Whiskey Tango Foxtrot,-10027861.0
52,Badlands,-395604.0
55,The Mean Season,-2650554.0
57,States of Grace,-596856.0
...,...,...
28440,One from the Heart,-25363204.0
28445,Explicit Ills,-1471436.0
28463,Area 51,-4992444.0
28472,Force of Execution,-7912878.0


In [ ]:
# Obter os filmes que possuem a palavra 'love' no título.

In [77]:
imdb_love = imdb[imdb["titulo"].str.contains("love", case = False)]

imdb.assign(
    tem_love = lambda x: x.titulo.str.lower().str.contains(r"\blove\b")
).query("tem_love")

,id_filme,titulo,ano,data_lancamento,generos,duracao,pais,idioma,orcamento,receita,...,nota_imdb,num_avaliacoes,direcao,roteiro,producao,elenco,descricao,num_criticas_publico,num_criticas_critica,tem_love
151,tt5164432,"Love, Simon",2018.0,2018-05-31,"Comedy, Drama, Romance",110,USA,"English, German, French",17000000.0,66316289.0,...,7.6,94565,Greg Berlanti,"Becky Albertalli, Elizabeth Berger",Fox 2000 Pictures,"Nick Robinson, Jennifer Garner, Josh Duhamel, ...",Simon Spier keeps a huge secret from his famil...,539.0,258.0,True
167,tt2106529,Lust for Love,2014.0,2014-02-07,"Comedy, Romance",85,USA,English,NaN,NaN,...,5.7,1425,Anton King,Anton King,AGGK Films,"Fran Kranz, Dichen Lachman, Beau Garrett, Cait...",Astor is dumped by Mila. He bumps into her ex ...,19.0,8.0,True
228,tt5457078,Love After Love,2017.0,2018-03-30,"Drama, Romance",91,USA,English,NaN,121098.0,...,5.4,850,Russell Harbaugh,"Russell Harbaugh, Eric Mendelsohn",Secret Engine,"Andie MacDowell, Chris O'Dowd, James Adomian, ...","Following the death of their father, two sons ...",27.0,26.0,True
461,tt0084293,Making Love,1982.0,1982-03-05,Drama,113,USA,English,NaN,11897978.0,...,6.8,2381,Arthur Hiller,"Barry Sandler, A. Scott Berg",IndieProd Company Productions,"Michael Ontkean, Kate Jackson, Harry Hamlin, W...",A successful young L.A. doctor and his equally...,65.0,15.0,True
489,tt0030256,I Met My Love Again,1938.0,1938-01-14,Romance,77,USA,English,428800.0,NaN,...,5.6,243,"Joshua Logan, Arthur Ripley","Allene Corliss, David Hertz",Walter Wanger Productions,"Joan Bennett, Henry Fonda, Louise Platt, Alan ...",College sweethearts Julie and Ives have planne...,6.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28199,tt1827578,The End of Love,2012.0,2012-01-21,Drama,89,USA,English,NaN,9342.0,...,5.9,1641,Mark Webber,Mark Webber,Poor Rich Kids,"Isaac Love, Mark Webber, Amanda Seyfried, Jaso...",When the mother of his infant son unexpectedly...,6.0,30.0,True
28240,tt0139499,No Ordinary Love,1994.0,2000-04-05,"Comedy, Romance",104,USA,English,NaN,NaN,...,4.4,303,Doug Witkins,Doug Witkins,Leo Productions,"Fred Blanco, Randy Brown, Angel Ceja, Jesus Co...",Several young adults live in a large house in ...,3.0,5.0,True
28266,tt0063242,Love Camp 7,1969.0,1969-10-04,"Drama, Thriller, War",96,USA,English,NaN,NaN,...,3.8,770,Lee Frost,Bob Cresse,NaN,"Bob Cresse, Maria Lease, Kathy Williams, Bruce...",Two female Army agents go undercover at a Nazi...,18.0,27.0,True
28270,tt2401023,Jack London's Love of Life,2012.0,2012-05-01,"Adventure, Drama, History",87,USA,English,225000.0,NaN,...,7.2,326,Kevin Swigert,"Robert Gregg, Jack London",Love of Life,"Pete Ammel, Phebe Horschel","Jack, an Alaska gold rush era miner, is abando...",9.0,NaN,True


In [ ]:
# Obter os files com gênero 'Drama' e ordernar pela quantidade de
# vezes que a palavra 'woman' (mulher) aparece na descrição.

In [80]:
def contar_string(x, string):
    return x.count(string)

imdb[imdb['generos'].str.contains("Drama", case = False)].assign(
    n_woman = imdb.descricao.str.lower().str.count("woman")
).sort_values("n_woman", ascending = False).filter(["titulo", "genero", "n_woman"]).dropna()

,titulo,n_woman
24278,Abigail Lesley Is Back in Town,3.0
25915,Body Fever,2.0
27692,From the Rough,2.0
26068,Break,2.0
19829,Queen Bee,2.0
...,...,...
9983,Back in the Day,0.0
9985,The Sentimental Engine Slayer,0.0
9986,Call Her Savage,0.0
9988,Murder in the Fleet,0.0


### Summarise

In [19]:
imdb.filter(["duracao", "orcamento"]).agg(["mean", "sum", "count"])

,duracao,orcamento
mean,9.305483e+01,1.213561e+07
sum,2.651132e+06,1.270598e+11
count,2.849000e+04,1.047000e+04


In [20]:
imdb.agg({
  "duracao": ["mean"],
  "orcamento": ["mean", "count"]
})

,duracao,orcamento
mean,93.054826,1.213561e+07
count,NaN,1.047000e+04


In [98]:
imdb.groupby(["ano"], as_index = False).agg(
  duracao_media = ("duracao", "mean"),
  desv_pad_duracao = ("duracao", "std"),
  receita_max = ("receita", max)
)

,ano,duracao_media,desv_pad_duracao,receita_max
0,1894.0,45.000000,NaN,NaN
1,1912.0,80.000000,28.284271,NaN
2,1913.0,88.000000,NaN,NaN
3,1914.0,73.312500,35.254728,NaN
4,1915.0,74.066667,34.900812,NaN
...,...,...,...,...
105,2016.0,93.359033,13.625927,1.153332e+09
106,2017.0,92.982320,13.029991,1.332540e+09
107,2018.0,93.658014,13.717957,2.048360e+09
108,2019.0,95.173104,18.143177,2.797801e+09


In [96]:
imdb.groupby('ano', as_index=False).apply(lambda x: pd.DataFrame({
    "duracao_media": [x.duracao.mean()],
    "desv_pad_duracao": [x.duracao.std()],
    "receita_max": [x.receita.max()],
    "lucro_medio": [(x.receita - x.orcamento).mean()]
}))

,,duracao_media,desv_pad_duracao,receita_max,lucro_medio
0,0,45.000000,NaN,NaN,NaN
1,0,80.000000,28.284271,NaN,NaN
2,0,88.000000,NaN,NaN,NaN
3,0,73.312500,35.254728,NaN,NaN
4,0,74.066667,34.900812,NaN,NaN
...,...,...,...,...,...
105,0,93.359033,13.625927,1.153332e+09,7.237195e+07
106,0,92.982320,13.029991,1.332540e+09,9.284472e+07
107,0,93.658014,13.717957,2.048360e+09,1.122318e+08
108,0,95.173104,18.143177,2.797801e+09,1.275660e+08


### Exercícios

In [ ]:
# Obtenha os 10 anos com as as maiores notas médias do IMDB.
# Coloque em um data frame ordeno pela média das notas.

In [ ]:
# Obtenha a proporção de filmes que deram prejuizo por ano.

### Left join

In [51]:
lucro_direcao = (imdb.
    assign(
        lucro = lambda x: x.receita - x.orcamento
    ).
    groupby(["direcao"], as_index = False).
    agg(
        lucro_medio = ('lucro', 'mean')
    )
)
lucro_direcao

,direcao,lucro_medio
0,'Evil' Ted Smith,NaN
1,'Philthy' Phil Phillips,NaN
2,A. Dean Bell,NaN
3,A. Edward Sutherland,NaN
4,"A. Edward Sutherland, John Rawlins",NaN
...,...,...
12453,Zoran Lisinac,-245414.0
12454,mink,NaN
12455,Édouard Molinaro,NaN
12456,Éric Rochat,NaN


In [59]:
imdb_com_lucro = pd.merge(imdb, lucro_direcao, how = "left", on = ["direcao"])

In [60]:
(imdb_com_lucro.
    assign(lucro = lambda x: x.receita - x.orcamento).
    filter(["titulo", "lucro", "lucro_medio"])
)

,titulo,lucro,lucro_medio
0,Prestige,NaN,NaN
1,Nob Hill,NaN,-2.966627e+06
2,The Shade,NaN,NaN
3,Viewer Discretion Advised,NaN,NaN
4,Broadcast News,47331309.0,4.774942e+07
...,...,...,...
28485,Jumanji: The Next Level,671575993.0,1.603053e+08
28486,Monster on the Campus,NaN,-7.474200e+05
28487,The Ward,-4656180.0,4.323364e+06
28488,A Game of Death,NaN,9.940914e+07
